In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
from pydub import AudioSegment

In [ ]:
labels = {'beach': 0,
          'bus': 1,
          'cafe/restaurant': 2,
          'car': 3,
          'city_center': 4,
          'forest_path': 5,
          'grocery_store': 6,
          'home': 7,
          'library': 8,
          'metro_station': 9,
          'office': 10,
          'park': 11,
          'residential_area': 12,
          'train': 13,
          'tram' : 14
          }

In [ ]:
FILEROOT = "audio/"
train_files = pd.read_table(os.path.join(FILEROOT, "test_files.txt"), names=['file', 'label'], sep='\s+') #dev.txt & train.txt
for i, afile in train_files.iterrows():
    sound = AudioSegment.from_file(afile.file)
    point = len(sound) // 12
    for i in range(10):
        divided_sound = sound[i*point: (i+2)*point] #subdivision se chevauchant
        divided_sound.export("test_divided/"+afile.file[6:-4]+"_"+str(i)+".wav", format="wav")

In [ ]:
FILEROOT = "audio_divided/"

labels_dict = np.load("divided_labels.npy").item()

train_files = pd.read_table(os.path.join("audio/train_divided.txt"), names=['file', 'label'], sep='\s+')
for i, afile in train_files.iterrows():
    for j in range(10):
        y, sr = librosa.load("audio_divided/"+str(afile.file[6:-4]+"_"+str(j)+".wav"), sr=None)
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=150).T,axis=0)
        if (i==0):
            X_train = [mfccs]
            y_train = [labels_dict[str(afile.file[6:-4]+"_"+str(j))]]
        else:
            X_train += [mfccs]
            y_train += [labels_dict[str(afile.file[6:-4]+"_"+str(j))]]
    if ((i+1) % 10 == 0 or i+1 == train_files.shape[0]):
        print("Train : {0}/{1}".format(i+1, train_files.shape[0]))

np.save("X_train_divided", X_train)
np.save("y_train_divided", y_train)

In [ ]:
FILEROOT = "audio_divided/"

X_test = []
labels_dict = np.load("divided_labels.npy").item()

test_files = pd.read_table(os.path.join("audio/test_files.txt"), names=['file', 'label'], sep='\s+')
for i, afile in test_files.iterrows():
    for j in range(10):
        y, sr = librosa.load("test_divided/"+str(afile.file[6:-4]+"_"+str(j)+".wav"), sr=None)
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=150).T,axis=0)
        if (len(X_test)==0):
            X_test = [mfccs]
        else:
            X_test += [mfccs]
    if ((i+1) % 10 == 0 or i+1 == test_files.shape[0]):
        print("Train : {0}/{1}".format(i+1, test_files.shape[0]))

np.save("X_test_divided", X_test)

In [ ]:
a = np.load("X_train_divided.npy")
a1 = np.load("y_train_divided.npy")
b = np.load("X_test_divided.npy")
print(a.shape, a1.shape, b.shape)